In [1]:
import random

def create_data_model(grid_size, num_locations, num_vehicles):
    """Generates the data for the problem with a grid and randomly placed locations."""
    data = {}
    
    # Generate the grid
    grid = [(x, y) for x in range(grid_size) for y in range(grid_size)]
    
    # Randomly select locations from the grid
    locations = random.sample(grid, num_locations)
    
    # Calculate distances between locations
    distance_matrix = []
    for i in range(num_locations):
        row = []
        for j in range(num_locations):
            if i == j:
                row.append(0)
            else:
                dist = abs(locations[i][0] - locations[j][0]) + abs(locations[i][1] - locations[j][1])
                row.append(dist)
        distance_matrix.append(row)
    
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = num_vehicles
    data['depot'] = 0  # Assuming the depot is located at index 0
    
    return data

In [2]:
"""Entry point of the program."""
# Instantiate the data problem.
data = create_data_model(500, 200, 4)

In [3]:
data['initial_routes'] = [[43, 176, 63, 40, 135, 99, 7, 12, 136, 118, 97, 28, 189, 164, 31, 182, 39, 139, 50, 115, 55, 27, 195, 48, 113, 77, 51, 109, 64, 194, 184, 15, 9, 105, 76, 133, 196, 191, 143, 151, 110, 171, 116, 100, 46, 168, 61, 102, 112, 120, 33, 124, 169], [96, 170, 65, 123, 70, 132, 193, 186, 117, 163], [90, 91, 103, 198, 197, 72, 2, 71, 128, 54, 157, 155, 52, 188, 6, 79, 23, 94, 49, 83, 181, 144, 146, 142, 174, 162, 45, 101, 147, 32, 114, 140, 53, 37, 159, 16, 11, 85, 199, 121, 80, 160, 167, 81, 56, 87, 69, 153, 68, 165, 74, 92, 60, 138, 150, 41, 19, 172, 75, 129, 84, 14, 17, 156, 20], [185, 1, 38, 22, 25, 190, 8, 10, 24, 58, 183, 47, 148, 104, 21, 175, 134, 95, 30, 177, 187, 126, 106, 73, 179, 62, 86, 98, 127, 180, 34, 158, 4, 107, 89, 36, 141, 42, 178, 161, 13, 18, 131, 82, 119, 66, 154, 192, 108, 78, 57, 149, 130, 35, 122, 145, 5, 44, 152, 93, 3, 111, 166, 59, 67, 125, 173, 88, 29, 137, 26]]

In [4]:
"""Vehicles Routing Problem (VRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

def convert_to_initial_solution(solution, manager, routing):
    initial_routes = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            if node_index != 0:
                route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        initial_routes.append(route)
    return initial_routes

def main():
    """Solve the CVRP problem."""

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        5000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Close model with the custom search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(10)
    # When an initial solution is given for search, the model will be closed with the default
    # search parameters unless it is explicitly closed with the custom search parameters.
    routing.CloseModelWithParameters(search_parameters)

    # Get initial solution from routes after closing the model.
    initial_solution = routing.ReadAssignmentFromRoutes(data['initial_routes'], True)

    # Solve the problem.
    solution = routing.SolveFromAssignmentWithParameters(
        initial_solution, search_parameters)

    # Print solution on console.
    if solution:
        print('Solution after search:')
        print_solution(data, manager, routing, solution)

        print(convert_to_initial_solution(solution, manager, routing))

import time

if __name__ == '__main__':
    start = time.time()
    main()
    print("Total time taken:", time.time()-start)

Solution after search:
Objective: 314670
Route for vehicle 0:
 0 ->  36 ->  80 ->  9 ->  55 ->  95 ->  124 ->  188 ->  30 ->  147 ->  44 ->  29 ->  133 ->  73 ->  169 ->  53 ->  10 ->  71 ->  162 ->  118 ->  115 ->  119 ->  102 ->  21 ->  183 ->  6 ->  26 ->  164 ->  174 ->  138 ->  168 ->  120 ->  25 ->  99 ->  75 ->  22 ->  88 ->  97 ->  2 ->  197 ->  101 ->  16 ->  61 ->  161 ->  70 ->  105 ->  113 ->  33 ->  107 ->  175 ->  117 ->  67 ->  31 ->  160 ->  57 ->  72 ->  38 ->  129 ->  14 ->  114 -> 0
Distance of the route: 3050m

Route for vehicle 1:
 0 ->  158 ->  11 ->  151 ->  127 ->  140 -> 0
Distance of the route: 584m

Route for vehicle 2:
 0 ->  108 ->  79 ->  63 ->  184 ->  135 ->  191 ->  173 ->  172 ->  19 ->  91 ->  84 ->  180 ->  154 ->  145 ->  60 ->  116 ->  193 ->  58 ->  69 ->  35 ->  112 ->  104 ->  199 ->  157 ->  143 ->  194 ->  178 ->  76 ->  96 ->  27 ->  136 ->  167 ->  94 ->  155 ->  195 ->  51 ->  196 ->  59 ->  52 ->  130 ->  146 ->  42 ->  128 ->  78 ->  23 -

In [5]:
# 346600
# 326462
# 310544